In [4]:
! pip install cvxopt

In [5]:
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important;} </style>"))

import numpy as np
import pandas as pd
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
import matplotlib.pyplot as plt

# Table of contents

* [I - Algorithms for quadratic programming ](#sec1)
    * [1. The active set method](#sec1_1)
* [II - Application of quadratic programming-SVM ](#sec_2)

# I - Algorithms for quadratic programming

The **quadratic programming(QP)** problems are linearly constrained optimization problems with a quadratic objective function.

$$ (QP)
\begin{cases}
\min_{x} f(x) := \min_{x}(\frac{1}{2}x^THx + c^Tx)\\
s.t. g(x): = Ax - b \geq0
\end{cases}
$$

Suppose $Q\in\mathbb{R^{n\times n}}$ is symmetric postive semidefinite, $C\in\mathbb{R^n}$, $A\in\mathbb{R^{n\times m}}$, $b\in\mathbb{R^{m}}$, with $A = (a_1^T, a_2^T, ... a_n^T), b=(b_1, b_2, ... b_n)$, and the feasible region $\mathcal{F} := \{x\in\mathbb{R^n}|Ax\geq b\}$, which can be further separated into inequality constraints $\mathcal{I} ,\;a_i^Tx-b_i\geq0$ and equality constraints $\mathcal{E},\;a_i^Tx - b_i = 0$


The **Karush-Kuhn-Tucker(KKT) condition** for the above QP problem is:

A point $x^* \in S $ is a minimiser to problem (QP) $\Longleftrightarrow$ $\exists $ with

$$
\begin{align}
(Stationarity)\quad &\nabla f(x^*) + \Sigma_i\lambda_i\nabla g(x^*) = Hx^*+c+\Sigma_i\lambda^Ta = 0 \\
(Primal\;feasibility)\quad &Inequality \quad a_i^Tx^*\geq b_i, i\in\mathcal{I}\\
&Equality\quad  a_i^Tx^*=0, i\in\mathcal{E}\\
(Dual\;feasibility) \quad &\lambda_i \geq 0, i \in \mathcal{I}\\
(Complementary\;slackness) \quad &\lambda_i(a^T_ix^*-b_i) = 0, i\in\mathcal{I}
\end{align}
$$

## I-1 The active set method for solving QP <a class='anchor' id='sec1'></a>

**Algorithm(Active set)**

- Initialization: Start with a feasible point $x^k$ of problem ($QP$)
- Start loop: mark step numerator k=0 and then iterate with k. Solve for the minimizer $\hat{x^k}$ of problem ($QP^k$)
$$
(QP^k) \begin{cases}
\min_{x} \frac{1}{2}x^THx + c^Tx\\
s.t. a_i^Tx - b_i = 0, i\in\mathcal{\mathcal{J_{k}}}
\end{cases}
$$
    - Case 1: $\hat{x^k} \in\mathcal{F}$, solve $\sum\limits_{i\in\mathcal{J_{k}}}\lambda_ia_i= -\nabla f(\hat{x_k})$
        - If all $\lambda_i \geq 0$, then minimum is found stop loop
        - Otherwise choose $s \in \mathcal{J_{k}}$ with $\lambda_s<0$ for example $\lambda_s=min{\lambda_i, j\in\mathcal{J_k}}$
          Set $\mathcal{J_{k+1}} = ActiveSet(\hat{x_k})-\{s\}$ (Desactivation step)
    - Case 2: $\hat{x^k} \notin\mathcal{F}$, for $d=\hat{x_k}-\hat{x_{k-1}}$ we determin a minimal $\alpha<0$ with $a_{i}^T(x+\alpha d) >= b_{i}$ and set $\hat{x_{k+1}} = \hat{x_k}+\alpha d$ 
    and $\mathcal{J_{k+1}}=ActiveSet(\hat{x_{k+1}})$ (Activation)

In [21]:
# Active set
# min 1/2x.THx+c.Tx
# s.t. Ax>=b

class Active_set(object):
    def __init__(self, H, C, A, b):
        """
        params: H, c, A, b defined as the same in QP problem, epsilon the tolerance to test positive value
        """
        self.H = H
        self.c = c
        self.A = A
        self.b = b
        self.epsilon = 1e-6

    def initial_set(self):
        """
        Initialize the active constraint set
        """
        # Choose the active contraint by simply taking the first
        activae_set_rows = [0]

        # Find the x0 that satisfy the first constraint in active set
        # Find the first coefficient in the chosen constraint that is not zero, let the element of x be b/coeff 
        # to satisfy the ax=b and set other element simply to be zero
        index = np.where(self.A[0] != 0)[0][0]
        value = self.A[0][index]
        feasible_x = np.zeros(len(self.A[0]))
        feasible_x[index] = self.b[0][0]/float(value)

        feasible_x = feasible_x.reshape(-1, 1)
        return activae_set_rows, feasible_x

    def calculate_delta(self, x):
        """
        Calculate the first order derivative at x
        """
        return np.matmul(self.H, x) + self.c

    def find_active_set(self):
        activate_set_rows, feasible_x = self.initial_set()
        steps = 0
        while True:
            steps += 1
            print("steps is {}".format(steps))

            # Calculate the derivative of f at current x, and the current active constraint set
            partial_x = self.calculate_delta(feasible_x)
            actual_A, actual_b, actual_b1 = find_new_data(self.A, self.b, activate_set_rows)
            # Solve for lagrangian for mu x = -delta f(x), with current active constraint
            delta = Lagrange(self.H, partial_x, actual_A, actual_b)
            solution = delta[0: self.H.shape[1]]
            # Case 1
            if np.sum(np.abs(solution)) < self.epsilon:
                outcome = Lagrange(self.H, self.c, actual_A, actual_b1)
                lambda_value = outcome[self.H.shape[1]:].flatten()
                min_value = lambda_value.min()
                # if all lambda > 0 
                if min_value >= 0:
                    print("Reach Optimization")
                    print("Optimize x is {}".format(feasible_x.flatten()))
                    print("Active set is {}".format(activate_set_rows))
                    print("lambda_value is {}".format(lambda_value))
                    print("\n")
                    break
                # Desactivation
                else:
                    index = np.argmin(lambda_value)
                    activate_set_rows.pop(index)
                    feasible_x = feasible_x
                    print("Not Reach Optimization")
                    print("Feasible x is {}".format(feasible_x.flatten()))
                    print("Active set is {}".format(activate_set_rows))
                    print("lambda_value is {}".format(lambda_value))
                    print("\n")
            # Case 2        
            else:
                in_row, alpha_k = self.calculate_alpha(feasible_x, solution.reshape(-1, 1), activate_set_rows)
                if in_row == -1:
                    alpha = 1
                else:
                    alpha = min(1, alpha_k)

                feasible_x += alpha * solution
                # Activation
                if alpha != 1:
                    activate_set_rows.append(in_row)
                    print("Not Reach Optimization")
                    print("Feasible x is {}".format(feasible_x.flatten()))
                    print("Active set is {}".format(activate_set_rows))
                    print("\n")
                    continue
                else:
                    outcome = Lagrange(self.H, self.c, actual_A, actual_b1)
                    lambda_value = outcome[self.H.shape[1]:].flatten()
                    min_value = lambda_value.min()
                    if min_value >= 0:
                        print("Reach Optimization")
                        print("Optimize x is {}".format(feasible_x.flatten()))
                        print("Active set is {}".format(activate_set_rows))
                        print("lambda_value is {}".format(lambda_value))
                        print("\n")
                        break
                    else:
                        index = np.argmin(lambda_value)
                        activate_set_rows.pop(index)
                        print("Not Reach Optimization")
                        print("Feasible x is {}".format(feasible_x))
                        print("Active set is {}".format(activate_set_rows))
                        print("lambda_value is {}".format(lambda_value))
                        print("\n")

    def calculate_alpha(self, x, d, activate_set_rows):
        """
        Calculate the step size param alpha
        """
        min_alpha = 0
        inrow = -1
        for i in range(self.A.shape[0]):
            if i in activate_set_rows:
                continue
            else:
                b_i = self.b[i][0]
                a_i = self.A[i].reshape(-1, 1)
                low_number = np.matmul(a_i.T, d)[0][0]
                if low_number >= 0:
                    continue
                else:
                    new_alpha = (b_i - np.matmul(a_i.T, x)[0][0])/float(low_number)
                    if inrow == -1:
                        inrow = i
                        min_alpha = new_alpha
                    elif new_alpha < min_alpha:
                        min_alpha = new_alpha
                        inrow = i
                    else:
                        continue
        return inrow, min_alpha


def Lagrange(H, c, A, b):
    # construct the lagrange matrix and solve it
    #   |  H  -A.T  |   |  wx  |   | -c |
    #   |           | * |      | = |   |
    #   |  A   0   |   |  wl  |   |-b  |
    up_layer = np.concatenate((H, -A.T), axis=1)
    zero_0 = np.zeros([A.shape[0], A.shape[0]])
    low_layer = np.concatenate((-A, zero_0), axis=1)
    lagrange_matrix = np.concatenate((up_layer, low_layer), axis=0)

    actual_b = np.concatenate((-c, -b), axis=0)
    lagrange_matrix_inverse = np.linalg.inv(lagrange_matrix)
    return np.matmul(lagrange_matrix_inverse, actual_b)


def find_new_data(A, b, activate_set_rows):
    # Update active constraint
    actual_A = A[activate_set_rows]
    actual_b = np.zeros_like(b[activate_set_rows])
    return actual_A, actual_b, b[activate_set_rows]


if __name__ == "__main__":
    
    """
    Example 5 of page 171 from chapter 4.2
    
    f(x) = x1^2 - x1x2 + x2^2 - 3x1
    constraints:
    -x1-x2>= -2
    x1 >= 0
    x2 >= 0
    -x1 >= -3/2
    
    thus we have
    H = np.array([ [2.0, -1], [-1, 2.0] ])
    c = np.array([ [-3.0], [0.0]]).reshape(-1,1)
    A = np.array([ [-1.0, -1.0], [1.0, 0],[0, 1],[-1.0, 0.0]])
    b = np.array( [ [-2.0], [0], [0], [-3/2] ] ).reshape(-1,1)
    
    The optimal is obtained at x = [x1, x2] = [1.5, 0.5]
    
    """
    
    H = np.array([ [2.0, -1], [-1, 2.0] ])
    c = np.array([ [-3.0], [0.0]]).reshape(-1,1)
    A = np.array([ [-1.0, -1.0], [1.0, 0],[0, 1],[-1.0, 0.0]])
    b = np.array( [ [-2.0], [0], [0], [-3/2] ] ).reshape(-1,1)

    test = Active_set(H, c, A, b)
    test.find_active_set()


steps is 1
Reach Optimization
Optimize x is [1.5 0.5]
Active set is [0]
lambda_value is [0.5]




# II - Application: SVM <a class='anchor' id='sec2'></a>

In a simpliest case of Support Vector Machine(SVM) to seperate points $x_i\in\mathbb{R}^{n}$ to value $y_i\in\{-1, 1\}$with a hyperplane described by  $<\omega, x>+\beta=0$, the condtion can be set as $<\omega, x_i>+\beta\geq1$ if $y_i=1$ and $<\omega,x_i>+\beta\leq-1$ if $y_i=-1$, $i\in\{1,...,m\}$. This classification problem can be described as an optimisation problem by finding the maximum 'margin' distance $2/\sqrt{<\omega, \omega>}$ between the two hyperplances $<\omega, x>+\beta=1$ and $<\omega, x>+\beta=-1$ then be formulated as

If the two classes are linearly separateble:
$$
(QP-linearly\;seperatable)=\begin{cases}
\min\limits_{\omega}\frac{1}{2}<\omega, \omega>\\
y_i(<\omega, x_i>+\beta)\geq1 (i=\{1,...m\})
\end{cases}
$$

If the two classes are not linearly separatable, we can introduce a nonnegative penalties $\xi_{i}$ as "soft" margin :
$$
(QP-no\;linearly\;seperatable) = \begin{cases}
\min\limits_{\omega}\frac{1}{2}<\omega, \omega> + C\sum\limits_{i=1}^{m}\xi_{i}\\
y_i(<\omega, x_i>+\beta)\geq1-\xi_{i} (i=\{1,...m\})
\end{cases}
$$




In [31]:
# Read breast-cancer-diagnostic csv files
import pandas as pd
data = pd.read_csv('breast-cancer-wisconsin.data', header=None)

# Adjust column names
headers = ['ID', 'Clump Thickness+A20', 'Uniformity of Cell Size', 'Uniformity of Cell Shape', 'Marginal Adhesion',
          'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin', 'Normal Nucleoli', 'Mistoses', 'Class']
data.columns = list(map(lambda x: x.replace(" ", '_'), headers))

# Data cleaning
data = data[data['Bare_Nuclei']!="?"].astype(int) # remove missing value lines
data = data.drop(["ID"], axis=1) # drop ID column
data['Class'] = np.where(data['Class']==2,1,-1) # Adjust labels

# Split data into training set and testing set
training_data = data[:120]
training_data_X = training_data.iloc[:,:9]
training_data_y = training_data.iloc[:,-1]

testing_data = data[120:]
testing_data_X = testing_data.iloc[:,:9]
testing_data_y = testing_data.iloc[:,-1]

# Initialize values and computing the input params
X = training_data_X.to_numpy()
y = training_data_y.to_numpy()

C = 1000
m, n = X.shape
y = y.reshape(-1,1)*1.
X_dash = y*X
H = np.dot(X_dash, X_dash.T)*1.

# Converting the input into cvxopt format
P = cvxopt_matrix(H)
q = cvxopt_matrix(-np.ones((m,1)))
G = cvxopt_matrix(np.vstack((np.eye(m)*-1, np.eye(m))))
h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m)*C)))
A = cvxopt_matrix(y.reshape(1,-1))
b = cvxopt_matrix(np.zeros(1))

# Run solver
sol = cvxopt_solvers.qp(P, q, G, h, A, b)
lambdas = np.array(sol['x'])

# Computing and printing params
w = ((y * lambdas).T @ X).reshape(-1,1)
S = (lambdas > 1e-4).flatten()
b = y[S] - np.dot(X[S], w)

# Display results
print('Lambdas = ', lambdas[lambdas > 1e-4], "\n")
print('w = ', w.flatten(), "\n")
print('b = ', b[0], "\n")


# Evaluate the performance of the classifier
def classifier(w, x, b):
    return np.sign(np.dot(w, x)+b)

# Set w and beta
W = w.flatten()
beta = b[0]


# Run classifier
testing_data['Prediction'] = testing_data_X.apply(lambda x: classifier(W,x,beta), axis=1).astype(int)

# -1 means positive of having cancer, +1 means negative (healty)
TP  = testing_data[(testing_data['Class'] == -1) & (testing_data['Prediction'] == -1)].shape[0]
TN  = testing_data[(testing_data['Class'] == 1) & (testing_data['Prediction'] == 1)].shape[0]
FP = testing_data[(testing_data['Class'] == 1) & (testing_data['Prediction'] == -1)].shape[0]
FN = testing_data[(testing_data['Class'] == -1) & (testing_data['Prediction'] == 1)].shape[0]

print("Precision : ", round(TP/(TP+FP)*100, 2), "%")
print("Recall : ", TP/(TP+FN)*100, "%")


     pcost       dcost       gap    pres   dres
 0:  1.6906e+03 -5.2157e+07  1e+08  5e-01  1e-10
 1:  3.7354e+04 -1.5377e+07  3e+07  8e-02  1e-10
 2:  7.5108e+04 -4.5746e+06  7e+06  2e-02  8e-11
 3:  5.7151e+04 -1.4167e+06  2e+06  4e-03  6e-11
 4:  4.3432e+03 -1.5789e+05  2e+05  1e-04  6e-11
 5: -3.5501e+03 -3.5288e+04  3e+04  1e-13  5e-11
 6: -3.7995e+03 -1.4052e+04  1e+04  2e-13  5e-11
 7: -4.2746e+03 -1.0660e+04  6e+03  8e-13  7e-11
 8: -4.3824e+03 -7.7619e+03  3e+03  9e-13  6e-11
 9: -4.7497e+03 -6.9636e+03  2e+03  2e-13  6e-11
10: -4.8098e+03 -6.6127e+03  2e+03  5e-13  7e-11
11: -5.2507e+03 -5.5924e+03  3e+02  3e-13  7e-11
12: -5.3545e+03 -5.3945e+03  4e+01  8e-13  8e-11
13: -5.3602e+03 -5.3820e+03  2e+01  4e-13  8e-11
14: -5.3618e+03 -5.3805e+03  2e+01  5e-13  8e-11
15: -5.3689e+03 -5.3692e+03  3e-01  5e-15  8e-11
16: -5.3690e+03 -5.3690e+03  3e-03  4e-13  9e-11
Optimal solution found.
Lambdas =  [9.99999998e+02 9.99999894e+02 9.24249058e-04 2.01519414e+02
 2.92835862e+02 1.29625

C:\Users\byhuang\AppData\Local\Temp\ipykernel_9120\970288415.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testing_data['Prediction'] = testing_data_X.apply(lambda x: classifier(W,x,beta), axis=1).astype(int)
